In [ ]:
# system imports
import os
from pathlib import Path

# Data Analysis
import pandas as pd
import pytimetk as tk
import polars as pl

# Files

import yaml


# QSConnect 
from qsconnect import Client

In [ ]:
# Constants
TODAY = "2025-07-28"

In [ ]:
TODAY = "2025-07-31"

In [ ]:
os.environ['FMP_API_KEY'] = yaml.safe_load(open('credentials.yml', encoding='utf-8'))['fmp_api_key']

In [ ]:
os.environ['FMP_API_KEY']

'EGUXlkKgIAY9jvTbGDXWb1WY4CAVfjgE'

In [ ]:
os.environ['QSCONNECT_ROOT'] = str(Path.cwd() / 'data' / 'qsconnect')

In [ ]:
os.environ['QSCONNECT_ROOT'] = str(Path.cwd() / 'data' / 'qsconnect-demo')

In [ ]:
client = Client()  # This will automatically create the data/qsconnect directory if it does not exist

In [ ]:
client

QSConnect(fmp_api_key=***WY4CAVfjgE, datalink_api_key=None)

In [ ]:
client.user_cache_dir

client.fmp_cache_dir

client.yahoo_cache_dir

# Database Directory
client.database_dir

PosixPath('/Users/brucebrownlee/Dev/GitHub/Resident/QS-Project/Clinic-05/data/qsconnect-demo/database')

In [ ]:
client.fmp_cache_dir

PosixPath('/Users/brucebrownlee/Dev/GitHub/Resident/QS-Project/Clinic-05/data/qsconnect-demo/cache/fmp')

In [ ]:
client.yahoo_cache_dir

PosixPath('/Users/brucebrownlee/Dev/GitHub/Resident/QS-Project/Clinic-05/data/qsconnect-demo/cache/yahoo')

In [ ]:
client.database_dir

PosixPath('/Users/brucebrownlee/Dev/GitHub/Resident/QS-Project/Clinic-05/data/qsconnect-demo/database')

In [ ]:
stock_list = client.stock_list(
    security_type="stock", 
    override_date=TODAY
)

2025-07-31 17:07:04,484 - QSConnect - WARNING - No cache file found for key 'stock_list-2025-07-31' at /Users/brucebrownlee/Dev/GitHub/Resident/QS-Project/Clinic-05/data/qsconnect-demo/cache/fmp/stock_list-2025-07-31.parquet
INFO [2025-07-31T17:07:07-0400][QSLog] Parsing response with content length: 16204860 bytes
INFO [2025-07-31T17:07:07-0400][QSLog] Successfully parsed response into a Pandas DataFrame.
2025-07-31 17:07:07,826 - QSConnect - INFO - Caching data at /Users/brucebrownlee/Dev/GitHub/Resident/QS-Project/Clinic-05/data/qsconnect-demo/cache/fmp
2025-07-31 17:07:07,871 - QSConnect - INFO - Cached data for key 'stock_list-2025-07-31' to /Users/brucebrownlee/Dev/GitHub/Resident/QS-Project/Clinic-05/data/qsconnect-demo/cache/fmp/stock_list-2025-07-31.parquet


In [ ]:
stock_list

stock_list.glimpse()

stock_list.shape
# stock_list.glimpse()

stock_list[stock_list['symbol'] == 'NVDA']

# Asset Types

stock_list['type'].value_counts()


# Exchange Types

stock_list[['exchange', 'exchangeShortName']].value_counts()


# Create a list of symbols to get prices for

<class 'pandas.core.frame.DataFrame'>: 85761 rows of 6 columns
symbol:             object            ['KZMS.ME', 'NOV.DE', '013A.F', 'B ...
name:               object            ['The Open Joint Stock Company Kra ...
price:              float64           [226.0, 41.365, 27.6, 12.92, 80.74 ...
exchange:           object            ['Moscow Stock Exchange', 'Deutsch ...
exchangeShortName:  object            ['MCX', 'XETRA', 'XETRA', 'XETRA', ...
type:               object            ['stock', 'stock', 'stock', 'stock ...


exchange                    exchangeShortName
Other OTC                   OTC                  13949
London Stock Exchange       LSE                   6805
NASDAQ                      NASDAQ                5323
New York Stock Exchange     NYSE                  4467
Bombay Stock Exchange       BSE                   4136
                                                 ...  
Stock Exchange of Thailand  SES                      1
Colombia Stock Exchange     BVC                      1
Ho Chi Minh Stock Exchange  HOSE                     1
NASDAQ Riga                 RIS                      1
NASDAQ Global Market        CBOE                     1
Name: count, Length: 79, dtype: int64

In [ ]:
stock_list[stock_list['symbol'] == 'NVDA']

,symbol,name,price,exchange,exchangeShortName,type
7365,NVDA,NVIDIA Corporation,177.87,NASDAQ Global Select,NASDAQ,stock


In [ ]:
stock_list['type'].value_counts()

type
stock    66150
etf      13138
trust     4267
fund      2206
Name: count, dtype: int64

In [ ]:
stock_list[['exchange', 'exchangeShortName']].value_counts()

exchange                    exchangeShortName
Other OTC                   OTC                  13949
London Stock Exchange       LSE                   6805
NASDAQ                      NASDAQ                5323
New York Stock Exchange     NYSE                  4467
Bombay Stock Exchange       BSE                   4136
                                                 ...  
Stock Exchange of Thailand  SES                      1
Colombia Stock Exchange     BVC                      1
Ho Chi Minh Stock Exchange  HOSE                     1
NASDAQ Riga                 RIS                      1
NASDAQ Global Market        CBOE                     1
Name: count, Length: 79, dtype: int64

In [ ]:
type(stock_list)

pandas.core.frame.DataFrame

In [ ]:
asset_types = ['stock'] 
exchange_types = ['NASDAQ', 'NYSE', 'AMEX']

symbols_to_get = []

for asset_type in asset_types:
    for exchange_type in exchange_types:
        filtered_symbols = stock_list[
            (stock_list['type'] == asset_type) & 
            (stock_list['exchangeShortName'] == exchange_type)
        ]['symbol'].tolist()
        symbols_to_get.extend(filtered_symbols)

symbols_to_get[0:10]  # Display the first 10 symbols

len(symbols_to_get)  # Total number of symbols to get prices for

# Add SPY and QQQ to the list for Benchmarking
symbols_to_get = ['SPY', 'QQQ'] + symbols_to_get

symbols_to_get

['SPY',
 'QQQ',
 'JANX',
 'BRID',
 'AARD',
 'HTCR',
 'BVS',
 'LE',
 'CMTL',
 'AXON',
 'AGNCO',
 'ALMS',
 'AKRO',
 'BHRB',
 'CLAR',
 'ATHE',
 'AVAH',
 'BOF',
 'CRMT',
 'FFIV',
 'HMST',
 'HST',
 'IBKR',
 'ICU',
 'BDMD',
 'ADV',
 'BPYPP',
 'JACK',
 'COO',
 'ANTA',
 'FBIZ',
 'LEXX',
 'FLNT',
 'GRVY',
 'ASND',
 'FRME',
 'CFFI',
 'IDCC',
 'HWCPZ',
 'LCFY',
 'QXO',
 'FLD',
 'IOBT',
 'LPTH',
 'FRSH',
 'FULT',
 'GH',
 'GLE',
 'MDAI',
 'CDW',
 'CCRN',
 'LRCX',
 'GOGL',
 'STEL',
 'VIRT',
 'HOLX',
 'IINN',
 'INOD',
 'IMTE',
 'IMXI',
 'JFBRW',
 'KROS',
 'LITM',
 'MCRB',
 'MBIO',
 'MFH',
 'LCFYW',
 'LCUT',
 'GOVX',
 'IBG',
 'LPTX',
 'JUNS',
 'AAOI',
 'ENTX',
 'EWBC',
 'CCB',
 'CBAN',
 'CHRW',
 'CHEF',
 'BWFG',
 'JD',
 'ASTI',
 'ENSC',
 'AMBR',
 'CRAI',
 'JOUT',
 'COYA',
 'ETOR',
 'AVBP',
 'FORR',
 'MMYT',
 'DDOG',
 'FWONA',
 'DSP',
 'HAS',
 'FUFU',
 'KALA',
 'ALKT',
 'FITBI',
 'GCT',
 'BETR',
 'AACG',
 'DOMH',
 'BSVN',
 'CHR',
 'CHCO',
 'DXLG',
 'ACET',
 'FAST',
 'LMFA',
 'IVVD',
 'LBTYA',
 'ARTL',


In [ ]:
asset_types = ['stock'] 
exchange_types = ['NASDAQ', 'NYSE', 'AMEX']

In [ ]:
exchange_types = ['NASDAQ', 'NYSE', 'AMEX']

In [ ]:
symbols_to_get = []

In [ ]:
asset_types = ['stock'] 
exchange_types = ['NASDAQ', 'NYSE', 'AMEX']

symbols_to_get = []

for asset_type in asset_types:
    for exchange_type in exchange_types:
        filtered_symbols = stock_list[
            (stock_list['type'] == asset_type) & 
            (stock_list['exchangeShortName'] == exchange_type)
        ]['symbol'].tolist()
        symbols_to_get.extend(filtered_symbols)

symbols_to_get[0:10]  # Display the first 10 symbols

len(symbols_to_get)  # Total number of symbols to get prices for

# Add SPY and QQQ to the list for Benchmarking
symbols_to_get = ['SPY', 'QQQ'] + symbols_to_get

symbols_to_get

['SPY',
 'QQQ',
 'JANX',
 'BRID',
 'AARD',
 'HTCR',
 'BVS',
 'LE',
 'CMTL',
 'AXON',
 'AGNCO',
 'ALMS',
 'AKRO',
 'BHRB',
 'CLAR',
 'ATHE',
 'AVAH',
 'BOF',
 'CRMT',
 'FFIV',
 'HMST',
 'HST',
 'IBKR',
 'ICU',
 'BDMD',
 'ADV',
 'BPYPP',
 'JACK',
 'COO',
 'ANTA',
 'FBIZ',
 'LEXX',
 'FLNT',
 'GRVY',
 'ASND',
 'FRME',
 'CFFI',
 'IDCC',
 'HWCPZ',
 'LCFY',
 'QXO',
 'FLD',
 'IOBT',
 'LPTH',
 'FRSH',
 'FULT',
 'GH',
 'GLE',
 'MDAI',
 'CDW',
 'CCRN',
 'LRCX',
 'GOGL',
 'STEL',
 'VIRT',
 'HOLX',
 'IINN',
 'INOD',
 'IMTE',
 'IMXI',
 'JFBRW',
 'KROS',
 'LITM',
 'MCRB',
 'MBIO',
 'MFH',
 'LCFYW',
 'LCUT',
 'GOVX',
 'IBG',
 'LPTX',
 'JUNS',
 'AAOI',
 'ENTX',
 'EWBC',
 'CCB',
 'CBAN',
 'CHRW',
 'CHEF',
 'BWFG',
 'JD',
 'ASTI',
 'ENSC',
 'AMBR',
 'CRAI',
 'JOUT',
 'COYA',
 'ETOR',
 'AVBP',
 'FORR',
 'MMYT',
 'DDOG',
 'FWONA',
 'DSP',
 'HAS',
 'FUFU',
 'KALA',
 'ALKT',
 'FITBI',
 'GCT',
 'BETR',
 'AACG',
 'DOMH',
 'BSVN',
 'CHR',
 'CHCO',
 'DXLG',
 'ACET',
 'FAST',
 'LMFA',
 'IVVD',
 'LBTYA',
 'ARTL',


In [ ]:
symbols_to_get[0:10]

['SPY', 'QQQ', 'JANX', 'BRID', 'AARD', 'HTCR', 'BVS', 'LE', 'CMTL', 'AXON']

In [ ]:
len(symbols_to_get)

11648

In [ ]:
symbols_to_get = ['SPY', 'QQQ'] + symbols_to_get

In [ ]:
symbols_to_get

['SPY',
 'QQQ',
 'SPY',
 'QQQ',
 'JANX',
 'BRID',
 'AARD',
 'HTCR',
 'BVS',
 'LE',
 'CMTL',
 'AXON',
 'AGNCO',
 'ALMS',
 'AKRO',
 'BHRB',
 'CLAR',
 'ATHE',
 'AVAH',
 'BOF',
 'CRMT',
 'FFIV',
 'HMST',
 'HST',
 'IBKR',
 'ICU',
 'BDMD',
 'ADV',
 'BPYPP',
 'JACK',
 'COO',
 'ANTA',
 'FBIZ',
 'LEXX',
 'FLNT',
 'GRVY',
 'ASND',
 'FRME',
 'CFFI',
 'IDCC',
 'HWCPZ',
 'LCFY',
 'QXO',
 'FLD',
 'IOBT',
 'LPTH',
 'FRSH',
 'FULT',
 'GH',
 'GLE',
 'MDAI',
 'CDW',
 'CCRN',
 'LRCX',
 'GOGL',
 'STEL',
 'VIRT',
 'HOLX',
 'IINN',
 'INOD',
 'IMTE',
 'IMXI',
 'JFBRW',
 'KROS',
 'LITM',
 'MCRB',
 'MBIO',
 'MFH',
 'LCFYW',
 'LCUT',
 'GOVX',
 'IBG',
 'LPTX',
 'JUNS',
 'AAOI',
 'ENTX',
 'EWBC',
 'CCB',
 'CBAN',
 'CHRW',
 'CHEF',
 'BWFG',
 'JD',
 'ASTI',
 'ENSC',
 'AMBR',
 'CRAI',
 'JOUT',
 'COYA',
 'ETOR',
 'AVBP',
 'FORR',
 'MMYT',
 'DDOG',
 'FWONA',
 'DSP',
 'HAS',
 'FUFU',
 'KALA',
 'ALKT',
 'FITBI',
 'GCT',
 'BETR',
 'AACG',
 'DOMH',
 'BSVN',
 'CHR',
 'CHCO',
 'DXLG',
 'ACET',
 'FAST',
 'LMFA',
 'IVVD',
 'L

In [ ]:
prices_path = client.historical_prices(
    symbols=symbols_to_get[:2000],
    start_date="2000-01-01",
    end_date=TODAY,
    api_calls_per_minute=2000, # Adjust based on your FMP plan
    cache=True,  # This will cache the file in the FMP cache directory
    return_path=True,  # This will return the path to the cached file instead of the DataFrame (which is useful for large datasets)
)

2025-07-31 17:24:09,370 - QSConnect - INFO - API calls limited to 2000 per 60 seconds.
Fetching data: 100%|██████████| 2000/2000 [00:46<00:00, 42.94symbol/s] 
2025-07-31 17:24:55,991 - QSConnect - INFO - Data fetching completed in 46.62 seconds


In [ ]:
prices_path =  "/Users/brucebrownlee/Dev/GitHub/Resident/QS-Project/Clinic-05/" + "data/qsconnect-demo/cache/fmp/historical-prices-2025-07-28.parquet"

In [ ]:
print(prices_path)

/Users/brucebrownlee/Dev/GitHub/Resident/QS-Project/Clinic-05/data/qsconnect-demo/cache/fmp/historical-prices-2025-07-28.parquet


In [ ]:
lazy_df = pl.scan_parquet(prices_path)

In [ ]:
lazy_df.head

<bound method LazyFrame.head of <LazyFrame at 0x361B2FF90>>

In [ ]:
# * Step 2: Get the stock prices for all tickers

# * FMP Prices API

# FMP API Notes: 
#  1. This will take 10 minutes to run as it fetches historical prices for 12,000+ assets and convert to a 3 GB parquet file.
#  2. You can adjust the `api_calls_per_minute` parameter based on your FMP plan to avoid hitting rate limits. My limit is 3000 calls per minute, so I set it to 2000 here because I saw I was getting ERRORS when I set it to 3000.

prices_path = client.historical_prices(
    symbols=symbols_to_get[:2000],
    start_date="2000-01-01",
    end_date=TODAY,
    api_calls_per_minute=2000, # Adjust based on your FMP plan
    cache=True,  # This will cache the file in the FMP cache directory
    return_path=True,  # This will return the path to the cached file instead of the DataFrame (which is useful for large datasets)
)

# Inspect the cached file using Polars Lazy Evaluation for large datasets

# * Your cached file path will be different, so you need to adjust this path accordingly

prices_path =  "/Users/brucebrownlee/Dev/GitHub/Resident/QS-Project/Clinic-05/" + "data/qsconnect-demo/cache/fmp/historical-prices-2025-07-28.parquet"

print(prices_path)

# Detect Missing Symbols

# Get the head of the DataFrame to see the first few rows
lazy_df = pl.scan_parquet(prices_path)

lazy_df.head()

lazy_df.head().collect().to_pandas()

# Get the unique symbols in the DataFrame
lazy_df = (
    pl.scan_parquet(prices_path)
    .select(pl.col("symbol").unique())
)   

unique_symbols = lazy_df.collect().to_pandas()

unique_symbols

# Difference between symbols_to_get and unique_symbols
missing_symbols = list(set(symbols_to_get) - set(unique_symbols['symbol'].tolist()))

pd.DataFrame(missing_symbols, columns=['symbol'])


# Locate a single stock ticker, e.g., Apple (AAPL)

lazy_df = (
    pl.scan_parquet(prices_path)
    .filter(pl.col("symbol") == "AAPL")
)

df = lazy_df.collect().to_pandas()

df

df.glimpse()

df \
    .groupby('symbol') \
    .plot_timeseries(
        date_column='date',
        value_column='close',
        title='Stock Price',
    )


# * Yahoo Finance API Downloads

# Note: This is a long running operation (about 10 minutes)

client.historical_prices_yahoo(
    symbols=symbols_to_get[:2000],  
    start_date="2000-01-01",
    end_date=TODAY,
    batch_size=100,
)


# * Step 3: Get the fundamental data for all tickers

# - WARNING: Long running operation (134 minutes)
# - Note: Must wait 10 seconds between bulk API requests

client.fetch_bulk_financial_statements(
    statement_type='all', # all available statements: income, balance, cash flow, ...
    periods='all', # annual and quarter 
    
    # start_year=2000,
    start_year=2024,  
    
    end_year=pd.to_datetime(TODAY).year,
    api_buffer_seconds=12,  # Adjust based on your FMP plan to avoid hitting rate limits
)

# * Step 4: Detect Cached Files

client.detect_cached_files()

client.summarize_cached_files()


# Detect Missing Cached Files

df = client.detect_missing_cached_files(
    statement_type="all",
    periods='all',
    start_year=2001,
    end_year=2025, # Try with 2025
    override_date=TODAY,
)

# Clean up Cached Files
help(client.delete_cached_files)

# client.delete_cached_files(df.iloc[[0]])

# * 2.0 BUILD YOUR QUANTAMENTAL FINANCIAL DATABASE ----


# * Step 1: Identify the files you want to include in your database

all_cached_files = client.detect_cached_files(
    # start_date=TODAY,
    # end_date=pd.to_datetime(TODAY) + pd.DateOffset(days=1),  
)

all_cached_files[all_cached_files['file_name'].str.startswith('historical-prices')]

all_cached_files[all_cached_files['file_name'].str.contains('2006')]

# * Step 2: Create a database from the cached files

# This takes a 5 minutes to run as it processes all the cached files and creates the qsconnect DuckDB database

errors_df = client.load_cached_files_to_database(
    cached_files_df=all_cached_files, 
    fresh=True, # Set to True to create a fresh database; set to False to append to an existing database
)

errors_df # If this is empty, then all files were loaded successfully; otherwise a DataFrame with errors will be returned

# * Step 3: Inspect the database

# Connect to the database
client.connect_to_database()

# List All Database Tables
client.list_database_tables()

# Glimpse All Database Tables
client.glimpse_all_database_tables(max_colwidth=25)

# Missing Values Report
client.missing_values_all_database_tables(max_colwidth=25)

# Missing Calendar Years Report for FMP Fundamental Data
client.missing_calendar_years_all_database_tables(
    start_year=2000, 
    end_year=2025,
)

# Quality Report
client.database_quality_report(
    start_year=2000, 
    end_year=2025
)

# Run SQL Queries on the Database
conn = client.connect_to_database()

conn.execute("""
    SELECT *
    FROM historical_prices_fmp
    LIMIT 5
""").fetchdf()

# Disconnect from the database
client.close_database_connection()

# DBCode Extension Demo
# - Use the DBCode extension to examine the database tables


# * 3.0 BUILD YOUR ZIPLINE BUNDLE ----

# * Step 1: Create a Zipline Bundle from the FMP Database

# Connect to the database
client.connect_to_database()

# Create a Zipline Bundle from the database
client.ingest_zipline_bundle_from_fmp_tables(bundle_name="qspro_demo_historical_prices_fmp")

# Disconnect from the database
client.close_database_connection()

# * Step 2: Open the Zipline hidden folder

# Mac users can find the hidden folder at:
#   1. Open Finder.
#   2. Press `Command + Shift + .` (period) to toggle hidden files visibility.
#   3. Navigate to the `~/.zipline` directory.

# Windows users can find the hidden folder at:
#   C:\Users\<YourUsername>\.zipline

2025-07-31 17:29:43,177 - QSConnect - INFO - Loaded cached data for the given date range from cache.


/Users/brucebrownlee/Dev/GitHub/Resident/QS-Project/Clinic-05/data/qsconnect-demo/cache/fmp/historical-prices-2025-07-28.parquet
<class 'pandas.core.frame.DataFrame'>: 6430 rows of 14 columns
date:              datetime64[ns]    [Timestamp('2025-07-28 00:00:00'),  ...
open:              float64           [214.03, 214.7, 213.9, 215.0, 213.1 ...
high:              float64           [214.85, 215.24, 215.69, 215.15, 21 ...
low:               float64           [213.06, 213.4, 213.53, 212.41, 212 ...
close:             float64           [214.05, 213.88, 213.76, 214.15, 21 ...
adjClose:          float64           [214.05, 213.88, 213.76, 214.15, 21 ...
volume:            float64           [36504158.0, 40268781.0, 46022620.0 ...
unadjustedVolume:  float64           [36504158.0, 40268781.0, 46022620.0 ...
change:            float64           [0.02, -0.82, -0.14, -0.85, 1.26, 0 ...
changePercent:     float64           [0.00934448, -0.38193, -0.06545115, ...
vwap:              float64           [

Fetching data batches:   0%|          | 0/20 [00:00<?, ?batch/s]ERROR [2025-07-31T17:30:08-0400][yfinance] 
2 Failed downloads:
ERROR [2025-07-31T17:30:08-0400][yfinance] ['JFBRW', 'LCFYW']: YFPricesMissingError('possibly delisted; no price data found  (1d 2000-01-01 -> 2025-07-31)')
Fetching data batches:   5%|▌         | 1/20 [00:22<07:06, 22.46s/batch]ERROR [2025-07-31T17:30:36-0400][yfinance] 
2 Failed downloads:
ERROR [2025-07-31T17:30:36-0400][yfinance] ['BCARU', 'DHCNL']: YFPricesMissingError('possibly delisted; no price data found  (1d 2000-01-01 -> 2025-07-31)')
Fetching data batches:  10%|█         | 2/20 [00:50<07:42, 25.71s/batch]ERROR [2025-07-31T17:31:02-0400][yfinance] 
2 Failed downloads:
ERROR [2025-07-31T17:31:02-0400][yfinance] ['MVSTW', 'BGLWW']: YFPricesMissingError('possibly delisted; no price data found  (1d 2000-01-01 -> 2025-07-31)')
Fetching data batches:  15%|█▌        | 3/20 [01:17<07:24, 26.14s/batch]ERROR [2025-07-31T17:31:36-0400][yfinance] Failed to get 

Help on method delete_cached_files in module qsconnect.client:

delete_cached_files(cached_files_df: pandas.core.frame.DataFrame) method of qsconnect.client.Client instance
    Delete cached files from the cache directory.
    
    Parameters
    ----------
    cached_files_df : pd.DataFrame
        DataFrame containing the cached files metadata (from `detect_cached_files` function).



Loading files (Batch 1):  50%|█████     | 5/10 [00:01<00:01,  3.09it/s]2025-07-31 17:52:04,761 - QSConnect - INFO - Data from bulk-cash-flow-statement-growth_2024_annual-2025-07-28.parquet inserted into table bulk_cash_flow_statement_growth_annual_fmp
2025-07-31 17:52:04,957 - QSConnect - INFO - Data from bulk-cash-flow-statement_2024_annual-2025-07-31.parquet inserted into table bulk_cash_flow_statement_annual_fmp
Loading files (Batch 1): 100%|██████████| 10/10 [00:03<00:00,  3.30it/s]
2025-07-31 17:52:06,100 - QSConnect - INFO - Processing batch 11-20 of 42
Loading files (Batch 2): 100%|██████████| 10/10 [00:03<00:00,  3.18it/s]
2025-07-31 17:52:09,243 - QSConnect - INFO - Processing batch 21-30 of 42
Loading files (Batch 3): 100%|██████████| 10/10 [00:02<00:00,  3.69it/s]
2025-07-31 17:52:11,951 - QSConnect - INFO - Processing batch 31-40 of 42
Loading files (Batch 4): 100%|██████████| 10/10 [00:02<00:00,  3.63it/s]
2025-07-31 17:52:14,705 - QSConnect - INFO - Processing batch 41-42



Table 1: 'bulk_balance_sheet_statement_annual_fmp'
Rows: 99337
Columns: 53
--------------------------
                     name         type        sample value
                   symbol      VARCHAR           000001.SZ
                     date TIMESTAMP_NS 2024-12-31 00:00:00
         reportedCurrency      VARCHAR                 CNY
                      cik       DOUBLE                 0.0
              fillingDate TIMESTAMP_NS 2024-12-31 00:00:00
             acceptedDate TIMESTAMP_NS 2024-12-31 00:00:00
             calendarYear       BIGINT                2024
                   period      VARCHAR                  FY
   cashAndCashEquivalents       DOUBLE      680935000000.0
     shortTermInvestments       DOUBLE                 0.0
cashAndShortTermInvest...       DOUBLE      680935000000.0
           netReceivables       DOUBLE                 0.0
                inventory       DOUBLE                 0.0
       otherCurrentAssets       DOUBLE                 0.0
       tota

2025-07-31 17:52:17,771 - QSConnect - INFO - Connected to DuckDB at /Users/brucebrownlee/Dev/GitHub/Resident/QS-Project/Clinic-05/data/qsconnect-demo/database/qsconnect.duckdb
2025-07-31 17:52:17,850 - QSConnect - INFO - Connection to DuckDB Database closed successfully: /Users/brucebrownlee/Dev/GitHub/Resident/QS-Project/Clinic-05/data/qsconnect-demo/database/qsconnect.duckdb
2025-07-31 17:52:17,858 - QSConnect - INFO - Connected to DuckDB at /Users/brucebrownlee/Dev/GitHub/Resident/QS-Project/Clinic-05/data/qsconnect-demo/database/qsconnect.duckdb


INFO [2025-07-31T17:52:22-0400][zipline.data.bundles.core] Ingesting qspro_demo_historical_prices_fmp
2025-07-31 17:52:22,846 - QSConnect - INFO - Initializing Data Loader.
2025-07-31 17:52:22,847 - QSConnect - INFO - Fetching data from database.
2025-07-31 17:52:22,854 - QSConnect - INFO - Processing 2469 unique symbols
2025-07-31 17:52:22,855 - QSConnect - INFO - Table schema validation passed.
2025-07-31 17:52:22,846 - QSConnect - INFO - Initializing Data Loader.
2025-07-31 17:52:22,847 - QSConnect - INFO - Fetching data from database.
2025-07-31 17:52:22,854 - QSConnect - INFO - Processing 2469 unique symbols
2025-07-31 17:52:22,855 - QSConnect - INFO - Table schema validation passed.
2025-07-31 17:52:23,292 - QSConnect - INFO - Validating data quality.
2025-07-31 17:52:23,500 - QSConnect - INFO - Data validation complete.
2025-07-31 17:52:23,512 - QSConnect - INFO - Generating asset metadata.
2025-07-31 17:52:23,825 - QSConnect - INFO - Writing asset metadata and exchanges.
2025-0